In [2]:
import os
import cv2
import time
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

# data augmentation
import albumentations as A
from albumentations.pytorch import ToTensorV2

# pretrained models
import torchvision
from torchvision import models, transforms

import os
import json
import glob
import random
import collections

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
def one_pass(model, dataloader, optimizer, lossFun, backwards=True, print_loss=False):
    
    if backwards == True:
        model.train()
    else:
        model.eval()
    
    total_loss = 0.0
    for x, y in tqdm(dataloader):
        
        y_pred = model(x)
        y_pred = torch.sigmoid(y_pred).squeeze()
        loss = lossFun(y_pred, y.float())
        total_loss += loss.item()
        
        if backwards == True:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    avg_loss = total_loss / len(dataloader)
    
    if print_loss == True:
        print(avg_loss)
    
    return avg_loss

def one_pass_acc(model, dataloader, num_points):
    model.eval()
    total_incorrect = 0
        
    for x, y in dataloader:
        y_pred = model(x)
        y_pred = torch.sigmoid(y_pred).squeeze()
        y_pred = (y_pred > 0.5).int()
        
        total_incorrect += torch.count_nonzero(y - y_pred).item()
        
    percent_wrong = total_incorrect / num_points
    return 1 - percent_wrong

In [4]:
class AnimalFacesDataset(Dataset):
    def __init__(self, df, augment=False):
        self.df = df
        self.augment = augment
                
        # define the transformation
        if augment == True:
            self.transforms = A.Compose([
                # spatial transforms
                A.RandomCrop(width=224, height=224),
                A.HorizontalFlip(p=.5),
                A.VerticalFlip(p=.5),
                A.Rotate(limit = 10, 
                         border_mode = cv2.BORDER_CONSTANT, 
                         value = 0.0, p = .75),
                
                # pixel-level transformation
                A.RandomBrightnessContrast(p=0.5),
                
                # we will normalize according to ImageNet since we will be using a pre-trained ResNet
                # this adjusts from [0,255] to [0,1]
                #A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                
                # convert to a tensor and move color channels
                ToTensorV2()
            ])
        else:
            self.transforms = A.Compose([
                # training/valid images have same size
                A.CenterCrop(width=224, height=224),
                
                # normalize
                #A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                
                # convert to a tensor and move color channels
                ToTensorV2()
            ])
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # get ingredients for retrieving image
        image_path = row['resized_location']
        
        # read the img
        img = np.load(image_path)
                
        # transform the image
        # certain transformations expect the uint8 datatype
        transformed = self.transforms(image=img.astype(np.uint8))
        img = transformed['image']
        img = img/255
        
        label = torch.tensor(row['MGMT_value'])
        
        return img, label

In [5]:
alexnet_frozen = models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [6]:
alexnet_frozen.features[0] = nn.Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))

In [7]:
alexnet_frozen

AlexNet(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [12]:
%%time
train_df = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
train_planes = pd.read_csv("../input/planes/train_cleaned.csv", index_col=0)
mid_pics = []
for patient in train_df.BraTS21ID:
    try:
        pic = round(train_planes[(train_planes.patient == patient) & 
                     (train_planes.modality == "T1w") &
                     (train_planes.plane == "axial")]["image_id"].median())
        mid_pics.append(pic)
    except:
        mid_pics.append(-1)

t1_train = train_df.copy()
t1_train["pic_id"] = mid_pics
t1_train = t1_train[t1_train.pic_id != -1]

CPU times: user 1min 4s, sys: 151 ms, total: 1min 4s
Wall time: 1min 4s


In [13]:
%%time
t1_train = t1_train.reset_index(drop=True)
t1_train["BraTS21ID"] = t1_train.BraTS21ID.astype(str).str.zfill(5)

def resize_img(image, resized_image_location, size):
    img = cv2.resize(image, size)
    np.save(resized_image_location, img)
    
def load_dicom(path):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

resized = []
for patient in t1_train.BraTS21ID:
    pic = t1_train[t1_train.BraTS21ID==patient].pic_id.values[0]
    
    original_image_location = f"../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{patient}/T1w/Image-{pic}.dcm"
    resized_image_location = f"./{patient}_{pic}.npy"
    resized.append(resized_image_location)
    
    image = load_dicom(original_image_location)
    resize_img(image, resized_image_location, (256, 256))

t1_train["resized_location"] = resized
t1_train.head()

CPU times: user 2.28 s, sys: 338 ms, total: 2.61 s
Wall time: 6.76 s


,BraTS21ID,MGMT_value,pic_id,resized_location
0,00000,1,17,./00000_17.npy
1,00002,1,16,./00002_16.npy
2,00003,0,17,./00003_17.npy
3,00005,1,14,./00005_14.npy
4,00006,1,16,./00006_16.npy


In [14]:
t1_train.tail(5)

,BraTS21ID,MGMT_value,pic_id,resized_location
562,01005,1,12,./01005_12.npy
563,01007,1,62,./01007_62.npy
564,01008,1,96,./01008_96.npy
565,01009,0,12,./01009_12.npy
566,01010,0,96,./01010_96.npy


In [15]:
t1_train = t1_train.reset_index(drop=True)

In [16]:
ds_train = AnimalFacesDataset(t1_train, augment=True)
dl_train = DataLoader(ds_train, batch_size=50, shuffle=False)

In [17]:
# to confirm everything was loaded correctly
print(len(ds_train))
for idx, (tensor, output) in enumerate(ds_train):
    pass

567


In [18]:
# turn off gradients for all the parameters in frozen
for param in alexnet_frozen.parameters():
            param.requires_grad = False

# re-intialize the last layer for our task
alexnet_frozen.classifier[6] = nn.Linear(4096, 1)

# pass the appropriate parameters to the optimizer
params_to_update = []

for param in alexnet_frozen.parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

optimizer = optim.Adam(params_to_update, lr=0.001)

In [19]:
next(iter(ds_train))[0]

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

In [20]:
y_pred = alexnet_frozen(next(iter(dl_train))[0])

In [21]:
lossFun = nn.BCELoss()

num_epochs = 100
train_losses = []

for epoch in tqdm(range(num_epochs)):
    print('Epoch: ', epoch)
    
    train_loss = one_pass(alexnet_frozen, dl_train, optimizer, lossFun)
    train_losses.append(train_loss)
    print('Train loss: ', train_loss)
        
    train_acc = one_pass_acc(alexnet_frozen, dl_train, len(ds_train))
    print('Train Acc: ', train_acc)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch:  0


  0%|          | 0/12 [00:00<?, ?it/s]

Train loss:  0.7218924711147944
Train Acc:  0.5291005291005291
Epoch:  1


  0%|          | 0/12 [00:00<?, ?it/s]

...

Train loss:  0.6921417266130447
Train Acc:  0.5749559082892417
Epoch:  98


  0%|          | 0/12 [00:00<?, ?it/s]

Train loss:  0.6936741272608439
Train Acc:  0.5837742504409171
Epoch:  99


  0%|          | 0/12 [00:00<?, ?it/s]

Train loss:  0.6998254905144373
Train Acc:  0.5714285714285714
